In [119]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [120]:
df = pd.read_csv("../data/curated/domain_final.csv")

In [121]:
df.loc[df['postcode'] == 3004, 'suburb_sold'] = 596
df.loc[df['postcode'] == 3004, 'suburb_avg_day'] = 147
df.loc[df['postcode'] == 3004, 'suburb_population'] = 6423.0
df.loc[df['postcode'] == 3004, 'suburb_age'] = 20
df.loc[df['postcode'] == 3004, 'suburb_owner'] = 30
df.loc[df['postcode'] == 3004, 'suburb_renter'] = 70
df.loc[df['postcode'] == 3004, 'suburb_family'] = 24
df.loc[df['postcode'] == 3004, 'suburb_single'] = 76

In [122]:
df = df.drop(['Unnamed: 0','uri','name','cost_text','desc','avg_income',
              'coordinates','school_title','SA2_CODE21',
              'school_distance','suburb_age'], axis=1)
df

,type,suburb_sold,suburb_avg_day,suburb_population,suburb_owner,suburb_renter,suburb_family,suburb_single,postcode,cost,...,dist_nearest_trainstation,dist_nearest_primary_school,dist_nearest_secondary_school,Beds,Bath,Parking,Median_tot_fam_inc_weekly,Tot_P_P,Median_age_persons,Median_rent_weekly
0,Apartment / Unit / Flat,596.0,147.0,47279.0,30.0,70.0,24.0,76.0,3000,500.0,...,1069.7,380.6,380.6,2.0,1.0,1,2248.0,9848.0,30.0,375.0
1,Apartment / Unit / Flat,387.0,120.0,47279.0,30.0,70.0,24.0,76.0,3000,570.0,...,614.1,491.7,355.7,1.0,1.0,0,2248.0,9848.0,30.0,375.0
2,Apartment / Unit / Flat,596.0,147.0,47279.0,30.0,70.0,24.0,76.0,3000,570.0,...,588.3,1635.2,703.0,2.0,1.0,1,2248.0,9848.0,30.0,375.0
3,Apartment / Unit / Flat,387.0,120.0,47279.0,30.0,70.0,24.0,76.0,3000,475.0,...,1345.1,584.0,584.0,1.0,1.0,1,2248.0,9848.0,30.0,375.0
4,Apartment / Unit / Flat,387.0,120.0,47279.0,30.0,70.0,24.0,76.0,3000,550.0,...,619.3,969.8,497.6,1.0,1.0,0,2248.0,9848.0,30.0,375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10153,House,81.0,49.0,5431.0,74.0,26.0,56.0,44.0,3996,400.0,...,89540.3,343.0,13431.1,3.0,1.0,2,1466.0,9446.0,54.0,250.0
10154,House,25.0,56.0,5431.0,74.0,26.0,56.0,44.0,3996,420.0,...,89954.5,1009.1,13845.3,2.0,1.0,0,1466.0,9446.0,54.0,250.0
10155,House,81.0,49.0,5431.0,74.0,26.0,56.0,44.0,3996,475.0,...,89143.3,35.3,13034.1,3.0,2.0,1,1466.0,9446.0,54.0,250.0
10156,House,81.0,49.0,5431.0,74.0,26.0,56.0,44.0,3996,400.0,...,89804.8,2704.1,13695.6,3.0,1.0,1,1466.0,9446.0,54.0,250.0


In [123]:
dummies = pd.get_dummies(df.postcode,prefix=['p'])
df = pd.concat([df,dummies],axis = 1)

dummies2 = pd.get_dummies(df.type)
df = pd.concat([df,dummies2],axis = 1)

dummies3 = pd.get_dummies(df.Beds,prefix=['bed'])
df = pd.concat([df,dummies3],axis = 1)

dummies4 = pd.get_dummies(df.Bath,prefix=['bath'])
df = pd.concat([df,dummies4],axis = 1)

dummies4 = pd.get_dummies(df.Parking,prefix=['park'])
df = pd.concat([df,dummies4],axis = 1)


df = df.drop(['Beds','Bath','Parking','postcode','type'], axis=1)







In [124]:
#df[df['suburb_population'].isnull()]['postcode'].value_counts()

In [125]:
#df.loc[(df['postcode'] == 3195)&(df['suburb_population'].isnull())]

In [126]:
df

,suburb_sold,suburb_avg_day,suburb_population,suburb_owner,suburb_renter,suburb_family,suburb_single,cost,dist_nearest_busstop,dist_nearest_trainstation,...,['park']_0,['park']_1,['park']_2,['park']_3,['park']_4,['park']_5,['park']_6,['park']_7,['park']_8,['park']_9
0,596.0,147.0,47279.0,30.0,70.0,24.0,76.0,500.0,117.9,1069.7,...,0,1,0,0,0,0,0,0,0,0
1,387.0,120.0,47279.0,30.0,70.0,24.0,76.0,570.0,351.3,614.1,...,1,0,0,0,0,0,0,0,0,0
2,596.0,147.0,47279.0,30.0,70.0,24.0,76.0,570.0,848.4,588.3,...,0,1,0,0,0,0,0,0,0,0
3,387.0,120.0,47279.0,30.0,70.0,24.0,76.0,475.0,247.2,1345.1,...,0,1,0,0,0,0,0,0,0,0
4,387.0,120.0,47279.0,30.0,70.0,24.0,76.0,550.0,507.0,619.3,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10153,81.0,49.0,5431.0,74.0,26.0,56.0,44.0,400.0,152.7,89540.3,...,0,0,1,0,0,0,0,0,0,0
10154,25.0,56.0,5431.0,74.0,26.0,56.0,44.0,420.0,779.4,89954.5,...,1,0,0,0,0,0,0,0,0,0
10155,81.0,49.0,5431.0,74.0,26.0,56.0,44.0,475.0,118.9,89143.3,...,0,1,0,0,0,0,0,0,0,0
10156,81.0,49.0,5431.0,74.0,26.0,56.0,44.0,400.0,1152.4,89804.8,...,0,1,0,0,0,0,0,0,0,0


In [127]:
for c in df.columns:
    if df[c].isnull().values.any():
        print(c)

In [128]:
df.isnull().sum()

suburb_sold          0
suburb_avg_day       0
suburb_population    0
suburb_owner         0
suburb_renter        0
                    ..
['park']_5           0
['park']_6           0
['park']_7           0
['park']_8           0
['park']_9           0
Length: 385, dtype: int64

In [129]:
target= np.array(df['cost'])
features = df.drop('cost', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

In [ ]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)
accuracies = []
for train_index, test_index in kf.split(features):

    data_train   = features[train_index]
    target_train = target[train_index]

    data_test    = features[test_index]
    target_test  = target[test_index]

    rf = RandomForestRegressor(n_estimators = 1000, 
                               random_state = 42, 
                               criterion='squared_error',
                               bootstrap=True)
    
    rf.fit(data_train, target_train)

    predictions = rf.predict(data_test)

    errors = abs(predictions - target_test)

    print('Mean Absolute Error:', round(np.mean(errors), 2))
    
    mape = 100 * (errors / target_test)
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print('Average accuracy:', average_accuracy)

In [ ]:

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(data_train, target_train)

print(rf_random.best_params_)

In [ ]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)
accuracies = []
for train_index, test_index in kf.split(features):

    data_train   = features[train_index]
    target_train = target[train_index]

    data_test    = features[test_index]
    target_test  = target[test_index]

    rf = RandomForestRegressor(
                               n_estimators = 600, 
                               min_samples_split = 5, 
                               min_samples_leaf = 1, 
                               max_features = 'sqrt', 
                               max_depth = 60, 
                               bootstrap = False)
    
    rf.fit(data_train, target_train)

    predictions = rf.predict(data_test)

    errors = abs(predictions - target_test)

    print('Mean Absolute Error:', round(np.mean(errors), 2))
    
    mape = 100 * (errors / target_test)
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print('Average accuracy:', average_accuracy)

In [ ]:
y = rf.feature_importances_
list_y = [a for a in y if a > 0.005]

list_y.sort()

list_y = list_y[:20]

list_of_index = []
for i in list_y:
    a = np.where(y==i)
    list_of_index.append(a)
    
index = []
for i in range(int(len(list_of_index))):
    a = list_of_index[i][0][0]
    index.append(a)

col = []
for i in feature_list:
    col.append(i)
labels = []
for i in index:
    b = col[i]
    labels.append(b)

In [ ]:
labels

In [ ]:
index